# Сбор данных

### Задание

Соберите отзывы (3-5 разных классов) о Россельхозбанке с сайта Банки.ру. Например, класс “Обслуживание” или “Кредит”.

Можно воспользоваться [ссылкой на сайт](https://www.banki.ru/services/responses/bank/rshb/).

Способ получения данных выберите самостоятельно — можете собирать вручную или написать код для парсинга данных.

Убедитесь, что вы имеете достаточно данных по каждому классу.
Чем больше отзывов вы соберете — тем лучше.

### Реализация

Для сбора данных будет реализован парсер для сайта banki.ru, собирающий все комментарии по некоторой услуге. Данные по комментариям будут собраны и соединены в единый DataFrame, затем сохранены в csv-формат. Каждому коммантарию будет присвоен свой класс.

Выберу классы: <br>
1. Дебетовая карта *(83 страницы)*
2. Ипотека *(117 страниц)*
3. Потребительский кредит *(53 страницы)*
4. Вклад *(74 страницы)*

Будут собраны следующие данные для каждого класса:
1. Заголовок
2. Текст отзыва
3. URL отзыва
4. Оценка
5. Дата
6. Класс {'debitcard', 'credit', 'deposit', 'hypothec'}

Для парсинга сайта будет использоваться библиотека Beautiful Soup, для запросов - requests.

In [3]:
# импортируем необходимые библиотеки
import requests 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [4]:
# Ссылки: 
# Потребительский кредит:
link_credits = 'https://www.banki.ru/services/responses/bank/rshb/product/credits/?page='
# Ипотека: 
link_hypothec = 'https://www.banki.ru/services/responses/bank/rshb/product/hypothec/?page='
# Дебетовая карта: 
link_debitcards = 'https://www.banki.ru/services/responses/bank/rshb/product/debitcards/?page='
# Вклад: 
link_deposits = 'https://www.banki.ru/services/responses/bank/rshb/product/deposits/?page='

In [5]:
# получить список словарей данных для страницы (уже прочитанной bs) и присвоить им класс label 
def get_info_for_bs_page(bs, label):
    responses = bs.find_all(class_='responses__item')
    result = [{
            'title': i.find('a').text,
            'url': 'banki.ru' + i.find('a')['href'],
            'text': i.find(class_='responses__item__message markup-inside-small markup-inside-small--bullet').text.strip(), 
            'score': int(i.find(class_='rating-grade').text.strip()) if i.find(class_='rating-grade') else None,
            'date': i.find('time', class_='display-inline-block').text,
            'label': label
    } for i in responses]
        
    return result

In [6]:
# получить все страницы по выбранной ссылке (link_reviews) с выбранным классом (label)
def get_all_reviews(link_reviews, label):
    # изменим заголовки, чтобы сайт не блокировал запросы
    session = requests.session() 
    session.headers['Accept'] = \
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    session.headers['User-Agent'] = \
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'
    # чтобы сайт не посчитал нас Ddos
    session.keep_alive = False

    all_reviews = []
    i = 1
    while 1:
        try:
            req = session.get(link_reviews + str(i))
        except:
            req = session.get(link_reviews + str(i))
        else: 
            if req.status_code == 404:
                break
            bs = BeautifulSoup(req.text)
            all_reviews.extend(get_info_for_bs_page(bs, label))
            print(f'Загружены данные {label}, для страницы {i}')
            i += 1
    return all_reviews

### Кредит

Загрузим все данные по кредиту с помощью функции с необходимыми параметрами:

In [7]:
all_credit = get_all_reviews(link_credits, 'credit')

Загружены данные credit, для страницы 1
Загружены данные credit, для страницы 2
Загружены данные credit, для страницы 3
Загружены данные credit, для страницы 4
Загружены данные credit, для страницы 5
Загружены данные credit, для страницы 6
Загружены данные credit, для страницы 7
Загружены данные credit, для страницы 8
Загружены данные credit, для страницы 9
Загружены данные credit, для страницы 10
Загружены данные credit, для страницы 11
Загружены данные credit, для страницы 12
Загружены данные credit, для страницы 13
Загружены данные credit, для страницы 14
Загружены данные credit, для страницы 15
Загружены данные credit, для страницы 16
Загружены данные credit, для страницы 17
Загружены данные credit, для страницы 18
Загружены данные credit, для страницы 19
Загружены данные credit, для страницы 20
Загружены данные credit, для страницы 21
Загружены данные credit, для страницы 22
Загружены данные credit, для страницы 23
Загружены данные credit, для страницы 24
Загружены данные credit, 

Сделаем из созданных данных датафрейм:

In [8]:
all_credit_df = pd.DataFrame(all_credit)

In [9]:
all_credit_df.head()

title  \
0                                      благодарность   
1  Числится задолженность по закрытым кредитам в ...   
2                        Досрочное погашение кредита   
3     Жуткий колхоз, незаконно повышают % по кредиту   
4                                     Хорошая работа   

                                                 url  \
0  banki.ru/services/responses/bank/response/1044...   
1  banki.ru/services/responses/bank/response/1044...   
2  banki.ru/services/responses/bank/response/1044...   
3  banki.ru/services/responses/bank/response/1044...   
4  banki.ru/services/responses/bank/response/1044...   

                                                text  score              date  \
0  ДОБРЫЙ ДЕНЬ !!!    ХОЧУ написать БЛАГОДАРНОСТЬ...    5.0  03.12.2020 13:53   
1  добрый день! подавал заявки на кредит в банки ...    NaN  02.12.2020 11:18   
2  Требование решить вопрос о корректном закрытии...    1.0   01.12.2020 0:13   
3  Осенью 2019 г. взял потребительский кредит.25....    1.0  30.11.2020 21:52   
4  Здравствуйте, хотел бы оставить отзыв о хороше...    5.0  30.11.2020 15:31   

    label  
0  credit  
1  credit  
2  credit  
3  credit  
4  credit

In [10]:
len(all_credit_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   1315 non-null   object 
 1   url     1315 non-null   object 
 2   text    1315 non-null   object 
 3   score   936 non-null    float64
 4   date    1315 non-null   object 
 5   label   1315 non-null   object 
dtypes: float64(1), object(5)
memory usage: 61.8+ KB


Повторим эту операцию для остальных классов. 

### Ипотека

In [11]:
all_hypothec = get_all_reviews(link_hypothec, 'hypothec')

Загружены данные hypothec, для страницы 1
Загружены данные hypothec, для страницы 2
Загружены данные hypothec, для страницы 3
Загружены данные hypothec, для страницы 4
Загружены данные hypothec, для страницы 5
Загружены данные hypothec, для страницы 6
Загружены данные hypothec, для страницы 7
Загружены данные hypothec, для страницы 8
Загружены данные hypothec, для страницы 9
Загружены данные hypothec, для страницы 10
Загружены данные hypothec, для страницы 11
Загружены данные hypothec, для страницы 12
Загружены данные hypothec, для страницы 13
Загружены данные hypothec, для страницы 14
Загружены данные hypothec, для страницы 15
Загружены данные hypothec, для страницы 16
Загружены данные hypothec, для страницы 17
Загружены данные hypothec, для страницы 18
Загружены данные hypothec, для страницы 19
Загружены данные hypothec, для страницы 20
Загружены данные hypothec, для страницы 21
Загружены данные hypothec, для страницы 22
Загружены данные hypothec, для страницы 23
Загружены данные hyp

In [12]:
all_hypothec_df = pd.DataFrame(all_hypothec)

In [13]:
all_hypothec_df.head()

title  \
0  Изменился график платежей и итоговая сумма про...   
1  Ставки повышают, документы теряют, удаленно ни...   
2                             Нет решения по ипотеке   
3                           Благодарность сотруднику   
4                                  Доступная ипотека   

                                                 url  \
0  banki.ru/services/responses/bank/response/1044...   
1  banki.ru/services/responses/bank/response/1044...   
2  banki.ru/services/responses/bank/response/1044...   
3  banki.ru/services/responses/bank/response/1044...   
4  banki.ru/services/responses/bank/response/1044...   

                                                text  score              date  \
0  Согласно кредитного ипотечного договора от 02....    1.0  05.12.2020 13:34   
1  Максимально ужасное обслуживание.Взял ипотеку ...    2.0  05.12.2020 13:25   
2  Добрый деньОставил заявку на ипотеку с господд...    1.0   05.12.2020 9:08   
3  Добрый день!👋 Я являюсь клиентом вашего банка,...    5.0  04.12.2020 21:21   
4  В середине сентябре 2020 я подала заявку на ип...    1.0  04.12.2020 18:16   

      label  
0  hypothec  
1  hypothec  
2  hypothec  
3  hypothec  
4  hypothec

In [ ]:
len(all_hypothec_df)

### Дебетовые карты

In [14]:
all_debitcards = get_all_reviews(link_debitcards, 'debitcard')

Загружены данные debitcard, для страницы 1
Загружены данные debitcard, для страницы 2
Загружены данные debitcard, для страницы 3
Загружены данные debitcard, для страницы 4
Загружены данные debitcard, для страницы 5
Загружены данные debitcard, для страницы 6
Загружены данные debitcard, для страницы 7
Загружены данные debitcard, для страницы 8
Загружены данные debitcard, для страницы 9
Загружены данные debitcard, для страницы 10
Загружены данные debitcard, для страницы 11
Загружены данные debitcard, для страницы 12
Загружены данные debitcard, для страницы 13
Загружены данные debitcard, для страницы 14
Загружены данные debitcard, для страницы 15
Загружены данные debitcard, для страницы 16
Загружены данные debitcard, для страницы 17
Загружены данные debitcard, для страницы 18
Загружены данные debitcard, для страницы 19
Загружены данные debitcard, для страницы 20
Загружены данные debitcard, для страницы 21
Загружены данные debitcard, для страницы 22
Загружены данные debitcard, для страницы 

In [15]:
all_debitcards_df = pd.DataFrame(all_debitcards)

In [16]:
all_debitcards_df.head()

title  \
0  Сотрудники банка разгласили мои персональные д...   
1                ПРОПОДАЮТ ПЛАТЕЖИ ЖКХ САХАЛИНЭНЕРГО   
2           Ужасное обслуживание зарплатного проекта   
3                           Не хотят закрывать карты   
4          Мутные балы программы лояльности "Урожай"   

                                                 url  \
0  banki.ru/services/responses/bank/response/1044...   
1  banki.ru/services/responses/bank/response/1044...   
2  banki.ru/services/responses/bank/response/1044...   
3  banki.ru/services/responses/bank/response/1044...   
4  banki.ru/services/responses/bank/response/1044...   

                                                text  score              date  \
0  В очередной раз уже пишу претензию на отвратит...    1.0  05.12.2020 15:14   
1  В МОБИЛЬНОМ ПРИЛОЖЕНИИ ОПЛАЧИВАЮ УСЛУГИ ЖКХ И ...    NaN  05.12.2020 11:40   
2  Я-госслужащий. Зарплатные проект обслуживается...    1.0  04.12.2020 22:43   
3  Первое обращение.30.09.2020 г.  по моей просьб...    1.0  04.12.2020 20:14   
4  В банках и магазинах уже давно прописаны довол...    1.0  04.12.2020 13:37   

       label  
0  debitcard  
1  debitcard  
2  debitcard  
3  debitcard  
4  debitcard

In [17]:
len(all_debitcards_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2048 entries, 0 to 2047
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   2048 non-null   object 
 1   url     2048 non-null   object 
 2   text    2048 non-null   object 
 3   score   1695 non-null   float64
 4   date    2048 non-null   object 
 5   label   2048 non-null   object 
dtypes: float64(1), object(5)
memory usage: 96.1+ KB


### Вклад

In [18]:
all_deposits = get_all_reviews(link_deposits, 'deposit')

Загружены данные deposit, для страницы 1
Загружены данные deposit, для страницы 2
Загружены данные deposit, для страницы 3
Загружены данные deposit, для страницы 4
Загружены данные deposit, для страницы 5
Загружены данные deposit, для страницы 6
Загружены данные deposit, для страницы 7
Загружены данные deposit, для страницы 8
Загружены данные deposit, для страницы 9
Загружены данные deposit, для страницы 10
Загружены данные deposit, для страницы 11
Загружены данные deposit, для страницы 12
Загружены данные deposit, для страницы 13
Загружены данные deposit, для страницы 14
Загружены данные deposit, для страницы 15
Загружены данные deposit, для страницы 16
Загружены данные deposit, для страницы 17
Загружены данные deposit, для страницы 18
Загружены данные deposit, для страницы 19
Загружены данные deposit, для страницы 20
Загружены данные deposit, для страницы 21
Загружены данные deposit, для страницы 22
Загружены данные deposit, для страницы 23
Загружены данные deposit, для страницы 24
З

In [19]:
all_deposits_df = pd.DataFrame(all_deposits)

In [20]:
all_deposits_df.head()

title  \
0                                       Ужасный банк   
1                                Не возвращают вклад   
2  Благодарность сотрудникам отделения и обращени...   
3                    Незаконно присвоили 600 рублей!   
4                        Отвратительное обслуживание   

                                                 url  \
0  banki.ru/services/responses/bank/response/1044...   
1  banki.ru/services/responses/bank/response/1044...   
2  banki.ru/services/responses/bank/response/1044...   
3  banki.ru/services/responses/bank/response/1044...   
4  banki.ru/services/responses/bank/response/1044...   

                                                text  score              date  \
0  Ужасный банк, полный непрофессионализм сотрудн...    2.0  05.12.2020 17:21   
1  У меня открыт вклад в отделении г. Череповец В...    1.0  03.12.2020 23:12   
2  Добрый день ! Я обращаюсь к руководству Москов...    5.0  03.12.2020 15:16   
3  Украли 600 рублей! Три года назад открыл у них...    1.0  03.12.2020 11:35   
4  Отвратительное обслуживание. Дозвониться до ну...    1.0  02.12.2020 19:28   

     label  
0  deposit  
1  deposit  
2  deposit  
3  deposit  
4  deposit

In [21]:
len(all_deposits_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832 entries, 0 to 1831
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   1832 non-null   object 
 1   url     1832 non-null   object 
 2   text    1832 non-null   object 
 3   score   1450 non-null   float64
 4   date    1832 non-null   object 
 5   label   1832 non-null   object 
dtypes: float64(1), object(5)
memory usage: 86.0+ KB


### Создание единого DataFrame и сохранение в csv

Тепреь создадим единый DataFrame:

In [22]:
reviews_df = pd.concat([all_credit_df, all_hypothec_df, all_debitcards_df, all_deposits_df])

In [23]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8113 entries, 0 to 1831
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   8113 non-null   object 
 1   url     8113 non-null   object 
 2   text    8113 non-null   object 
 3   score   6359 non-null   float64
 4   date    8113 non-null   object 
 5   label   8113 non-null   object 
dtypes: float64(1), object(5)
memory usage: 443.7+ KB


Сохраним созданный df в формате csv (без индекса): 

In [24]:
reviews_df.to_csv('rshb_reviews.csv', index=False)

Таким образом, были получены все отзывы по четырем классам услуг с сайта banki.ru. При необходимости можно расширить выборку, добавив ссылку и label для нового класса. 